In [ ]:
import cv2
import os
import numpy as np
import pyautogui as pauto
from PIL import Image,ImageStat

In [ ]:
facecascPath=os.path.dirname(cv2.__file__)+"/data/haarcascade_frontalface_default.xml"
eyecascPath=os.path.dirname(cv2.__file__)+"/data/haarcascade_eye.xml"
faceCascade = cv2.CascadeClassifier(facecascPath)
eye_cascade = cv2.CascadeClassifier(eyecascPath)

In [ ]:
def detect_faces(img, cascade):
    gray_frame = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    coords = cascade.detectMultiScale(gray_frame,scaleFactor=1.1,minNeighbors=5,flags=cv2.CASCADE_SCALE_IMAGE)
    if len(coords) > 1:
        biggest = (0, 0, 0, 0)
        for i in coords:
            if i[3] > biggest[3]:
                biggest = i
        biggest = np.array([i], np.int32)
    elif len(coords) == 1:
        biggest = coords
    else:
        return []
    return biggest

In [ ]:
def detect_eyes(img, cascade):
    gray_frame = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    eyes = cascade.detectMultiScale(gray_frame,scaleFactor=1.1,minNeighbors=5,flags=cv2.CASCADE_SCALE_IMAGE) # detect eyes
    width = np.size(gray_frame, 1) # get face frame width
    height = np.size(gray_frame, 0) # get face frame height
    left,right = (None,None)
    for (x, y, w, h) in eyes:
        if y > height / 2:
            pass
        eyecenter = x + w / 2  # get the eye center
        if eyecenter < width * 0.5:
            left = (y,y + h, x,x + w)
        else:
            right = (y,y + h, x,x + w)
    return left, right

In [ ]:
def cut_eyebrows(img):
    height, width = img.shape[:2]
    eyebrow_h = int(height / 4)
    img = img[eyebrow_h:height, 0:width]
    return img

In [ ]:
def blob_process(img, detector):
    keypoints = detector.detect(img)
    return keypoints

In [ ]:
def mouse_index(x,y):
    return (x,y)


In [ ]:
def brightness(im_file):
    im = Image.fromarray(im_file).convert('L')
    stat = ImageStat.Stat(im)
    return stat.rms[0]

In [ ]:
# video_capture = cv2.VideoCapture(0)

# rows,cols = pauto.size()
# pauto.moveTo(rows/2,cols/2)
# try:
#     ret, img = video_capture.read()
#     box = detect_faces(img,faceCascade)
#     cx_ = (box[0][3] + box[0][1])/2
#     cy_ = (box[0][3] + box[0][1])/2
#     cx = cx_
#     cy = cy_
    
#     MIN_MOVE=10
#     while True:
#         ret, img = video_capture.read()
#         gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    
#         val = (int)(brightness(img))
    
#         faces = detect_faces(img,faceCascade)
    
#         try:
#             for (p,q,r,s) in faces:
#                 x_mean = (p+(p+r))/2
#                 y_mean = (q+(q+s))/2
            
#                 face_gray = gray_img[q:q+s, p:p+r]
#                 face_color = img[q:q+s, p:p+r]
                        
#                 mouse = mouse_index(int(x_mean),int(y_mean))
#                 cv2.circle(img,mouse, radius=5, color=(127,0,0), thickness=-1)            
            
#                 (left_eye,right_eye) = detect_eyes(face_color,eye_cascade)
#                 if(right_eye!=None):
#                     (hx,hq,hr,hs) = right_eye
#                     eye_gray = face_gray[hx:hq,hr:hs]
#                     eye_roi = cv2.GaussianBlur(eye_gray, (7, 7), 0)
#                     eye_color = face_color[hx:hq,hr:hs]
                
#                     _, threshold = cv2.threshold(eye_roi,35, 255, cv2.THRESH_BINARY_INV)
#                     threshold = cut_eyebrows(threshold)
            
#                     contours, _ = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#                     contours = sorted(contours, key=lambda x: cv2.contourArea(x), reverse=True)
            
#                     for cnt in contours:
#                         (xu, yu, wu, hu) = cv2.boundingRect(cnt)
#                         eye_xmean = (int)((2*xu+wu)/2)
#                         eye_ymean = (int)((2*yu+hu)/2)
#                         cv2.circle(eye_color,(eye_xmean,eye_ymean), radius=5, color=(100,0,0), thickness=-1)
                    
#                         cx,cy = (eye_xmean,eye_ymean)
#                         if abs(cx-cx_) > abs(cy-cy_):
#                             if cx - cx_ > MIN_MOVE:
#                                 print('LEFT')
#                             elif cx - cx_ < -MIN_MOVE:
#                                 print('RIHT')
#                         else:
#                             if cy - cy_ > MIN_MOVE:
#                                 print('DOWN')
#                                 pauto.scroll(-10)
#                             elif cy - cy_ < -MIN_MOVE:
#                                 print('UP')
#                                 pauto.scroll(10)
#                         break
#         except Exception as e:
#             print(str(e))

#         rimg = cv2.flip(img, 1)
#         cv2.imshow("Video", rimg)

#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break
#         cx_ = cx
#         cy_ = cy
# except Exception as e:
#     print(str(e))
# video_capture.release()
# cv2.destroyAllWindows()

In [ ]:
# os.path.dirname(dlib.__file__)+"\shape_predictor_68_face_landmarks.dat"

In [ ]:
import cv2
import dlib
import os
import pyautogui as pauto
import numpy as np
    
def shape_to_np(shape, dtype="int"):
    coords = np.zeros((68, 2), dtype=dtype)
    for i in range(0, 68):
        coords[i] = (shape.part(i).x, shape.part(i).y)
    return coords

def eye_on_mask(mask, side):
    points = [shape[i] for i in side]
    points = np.array(points, dtype=np.int32)
    mask = cv2.fillConvexPoly(mask, points, 255)
    return mask

def contouring(thresh, mid, img, right=False):
    cnts, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    try:
        cnt = max(cnts, key = cv2.contourArea)
        M = cv2.moments(cnt)
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
        if right:
            cx += mid
        cv2.circle(img, (cx, cy), 4, (0, 0, 255), 2)
        return (cx,cy)
    except:
        return ()

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(os.path.dirname(dlib.__file__)+"\shredictor_68_face_landmarks.dat"ape_p)

left = [36, 37, 38, 39, 40, 41]
right = [42, 43, 44, 45, 46, 47]

cap = cv2.VideoCapture(0)
ret, img = cap.read()
thresh = img.copy()

cv2.namedWindow('image')
kernel = np.ones((9, 9), np.uint8)

def nothing(x):
    pass
cv2.createTrackbar('threshold', 'image', 0, 255, nothing)


cx = 0
cy = 0
    
MIN_MOVE=1

while(True):
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 1)
    for rect in rects:
        shape = predictor(gray, rect)
        shape = shape_to_np(shape)
        mask = np.zeros(img.shape[:2], dtype=np.uint8)
        mask = eye_on_mask(mask, left)
        mask = eye_on_mask(mask, right)
        mask = cv2.dilate(mask, kernel, 5)
        eyes = cv2.bitwise_and(img, img, mask=mask)
        mask = (eyes == [0, 0, 0]).all(axis=2)
        eyes[mask] = [255, 255, 255]
        mid = (shape[42][0] + shape[39][0]) // 2
        eyes_gray = cv2.cvtColor(eyes, cv2.COLOR_BGR2GRAY)
        threshold = cv2.getTrackbarPos('threshold', 'image')
        _, thresh = cv2.threshold(eyes_gray, threshold, 255, cv2.THRESH_BINARY)
        thresh = cv2.erode(thresh, None, iterations=2) #1
        thresh = cv2.dilate(thresh, None, iterations=4) #2
        thresh = cv2.medianBlur(thresh, 3) #3
        thresh = cv2.bitwise_not(thresh)
        v = contouring(thresh[:, 0:mid], mid, img)
        v9 = contouring(thresh[:, mid:], mid, img, True)
        if(len(v)>0):
            if(cx==0 and cy==0):
                cx = v[0]
                cy = v[1]
            else:
                cx_ = v[0]
                cy_ = v[1]
                if abs(cx-cx_) > abs(cy-cy_):
                    if cx - cx_ > MIN_MOVE:
                        print('Left')
                        pauto.scroll()
                    elif cx - cx_ < -MIN_MOVE:
                        print('RIHT')
                else:
                    if  (cy - cy_ < -MIN_MOVE):
                        print("UP")
                        pauto.scroll(100)
                    elif cy - cy_ > MIN_MOVE:
                        print('DOWN')
                        pauto.scroll(-100)
                        
#                     :
#                     elif:
#                         print('UP')
#                         pauto.scroll(100)
            cx_ = cx
            cy_ = cy
    cv2.imshow('eyes', img)
    cv2.imshow("image", thresh)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

Left
Left
UP
UP
UP
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
UP
UP
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
UP
UP
RIHT
RIHT
RIHT
RIHT
UP
UP
UP
UP
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
UP
UP
RIHT
RIHT
RIHT
RIHT
RIHT
RIHT
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
Left
L